In [26]:
from pathlib import Path
import os
os.getcwd()

'/home/sakamoto/spectrum_re/note/data_prepare'

In [27]:
import sys
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, module_path)

import note.base as base
sys.path.pop(0)

SEED = 42
base.fix_seed(SEED)

In [28]:
import zipfile

zip_path = Path('../../data/basedata/height_tabledata.zip')
extract_to = Path('../../data/basedata/height_tabledata')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [29]:
extract_to = Path('../../data/basedata/height_tabledata')
for path in list(extract_to.rglob('*.txt')):
    dataframe = base.loadDataframe(str(path))
    base.insertdata(dataframe, path='../../data/basedata/ms2_lipid_data.db')

In [30]:
import pandas as pd

df = base.import_sqlite3_data('../../data/basedata/ms2_lipid_data.db').drop_duplicates().reset_index(drop=True)
df['dataset'] = pd.factorize(df['dataset'])[0]

df.to_csv('../../data/basedata/ms2_lipiddata.csv', index=False)

In [7]:
df = df[df['Comment'].str.lower() == "ok"] 
df = df[~df['Metabolitename'].str.contains('\(d')]

df.to_csv('../../data/basedata/df_filter_ok.csv')

In [8]:
df_pos = df[df['Adducttype'].str.contains('\]\+') | df['Adducttype'].str.contains('\]2\+')]
df_neg = df[df['Adducttype'].str.contains('\]\-') | df['Adducttype'].str.contains('\]2\-')]
print(len(df), len(df_pos)+len(df_neg), len(df_pos), len(df_neg))

18238 18238 8574 9664


In [8]:
df_pos.to_csv('../../data/basedata/df_pos.csv')
df_neg.to_csv('../../data/basedata/df_neg.csv')

In [9]:
df_pos_exp_ = base.spectrum_tidy(df_pos)
df_pos_neuloss_ = base.neutralloss_tidy(df_pos)
df_pos_data = base.spectrum_neutralloss_table(df_pos_exp_, df_pos_neuloss_)#, df_pos_remainder_)

df_pos_data_ = pd.merge(df_pos_data, df[['Metabolitename','Ontology','dataset','AlignmentID','AverageMz']],\
                        on=['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'], how='left')
df_pos_data_ = df_pos_data_[~df_pos_data_['Metabolitename'].str.contains("also known as")]. \
               set_index(['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'])

In [10]:
df_neg_exp_ = base.spectrum_tidy(df_neg)
df_neg_neuloss_ = base.neutralloss_tidy(df_neg)
df_neg_data = base.spectrum_neutralloss_table(df_neg_exp_, df_neg_neuloss_)

df_neg_data_ = pd.merge(df_neg_data, df[['Metabolitename','Ontology','dataset','AlignmentID','AverageMz']],\
                        on=['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'], how='left')
df_neg_data_ = df_neg_data_[~df_neg_data_['Metabolitename'].str.contains("also known as")]. \
               set_index(['Metabolitename','Ontology','dataset','AlignmentID','AverageMz'])

In [11]:
df_pos_data_.to_csv('../../data/basedata/df_pos_wide.csv')
df_neg_data_.to_csv('../../data/basedata/df_neg_wide.csv')